In [1]:
import os

In [2]:
%pwd

'c:\\Users\\laptop\\Desktop\\e2e_aws_mlops_pipeline\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\laptop\\Desktop\\e2e_aws_mlops_pipeline'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [6]:
from src.datascience_project.constants import *
from src.datascience_project.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                config_file_path=CONFIG_FILE_PATH,
                params_file_path=PARAMS_FILE_PATH,
                schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
from src.datascience_project import logger
import zipfile

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            
            filename, headers = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"{filename} downloaded! info: \n{headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists. Skipping download.")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        if not os.path.exists(unzip_path):
            os.makedirs(unzip_path)
            #logger.info(f"Created directory: {unzip_path}")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")
            


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-08-02 15:30:51,829]: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-02 15:30:51,831]: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-02 15:30:51,833]: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-02 15:30:51,834]: INFO: common: created directory at: artifacts]
[2025-08-02 15:30:51,834]: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-02 15:30:51,838]: INFO: 825335496: File artifacts\data_ingestion\data.zip already exists. Skipping download.]
[2025-08-02 15:30:51,842]: INFO: 825335496: Extracted files to artifacts\data_ingestion]
